# Coronavirus is a family of viruses that are named after their spiky crown. The novel coronavirus, also known as SARS-CoV-2, is a contagious respiratory virus that first reported in Wuhan, China. On 2/11/2020, the World Health Organization designated the name COVID-19 for the disease caused by the novel coronavirus. This notebook aims at exploring COVID-19 through data analysis and projections.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly_express as px
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff
%matplotlib inline
import datetime as dt

In [ ]:
# import data

covid_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
covid_data.head()

In [ ]:
confirmed_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
recoverd_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
death_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')

In [ ]:
# let's check size and info of the data

print("Size/Shape of the dataset: ",covid_data.shape)
print("Info of the dataset: ",covid_data.info())

Let's check for missing values in data

In [ ]:
total_of_all = covid_data.isnull().sum().sort_values(ascending=False)
percent_of_all = (covid_data.isnull().sum()/covid_data.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total_of_all, percent_of_all], axis=1, keys=['Total', 'Percent'])
missing_data_test.head()

Let's handle missing data

In [ ]:
# drop the column SNo as we don't need it

covid_data = covid_data.drop(columns=['SNo'])

In [ ]:
covid_data['Province/State'] = covid_data['Province/State'].fillna('Not known')

In [ ]:
#Converting "Observation Date" into Datetime format

covid_data["ObservationDate"] = pd.to_datetime(covid_data["ObservationDate"])

**Worldwide analysis**

In [ ]:
confirmed = covid_data.groupby('ObservationDate').sum()['Confirmed'].reset_index()
deaths = covid_data.groupby('ObservationDate').sum()['Deaths'].reset_index()
recovered = covid_data.groupby('ObservationDate').sum()['Recovered'].reset_index()

In [ ]:
trace1 = go.Bar(x = confirmed['ObservationDate'],
              y = confirmed['Confirmed'],
              name = 'Confirmed',
              marker_color = 'blue')

trace2 = go.Bar(x = deaths['ObservationDate'],
              y = deaths['Deaths'],
              name = 'Death',
              marker_color = 'red')

trace3 = go.Bar(x = recovered['ObservationDate'],
              y = recovered['Recovered'],
              name = 'Recovered',
              marker_color = 'green')

layout = go.Layout(barmode = 'group',
                  bargap = 0.15,
                  bargroupgap = 0.1,
                  title = "Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
)

data = [trace1, trace2, trace3]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
trace1 = go.Scatter(x = confirmed['ObservationDate'],
              y = confirmed['Confirmed'],
              name = 'Confirmed',
              mode='lines+markers',
              line=dict(color='blue', width=1))

trace2 = go.Scatter(x = deaths['ObservationDate'],
              y = deaths['Deaths'],
              name = 'Death',
              mode='lines+markers',
              line=dict(color='red', width=1))

trace3 = go.Scatter(x = recovered['ObservationDate'],
              y = recovered['Recovered'],
              name = 'Recovered',
              mode='lines+markers',
              line=dict(color='green', width=1))

layout = go.Layout(title = "Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Line Chart)",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
)

data = [trace1, trace2, trace3]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

# **Each Country Analysis**

In [ ]:
def country_data(country_name):
    country_data = covid_data[covid_data['Country/Region']==country_name]
    return country_data

def country_data_datewise(country):
    country_datewise = country.groupby(["ObservationDate"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})
    return country_datewise

In [ ]:
def country_all_data(country_data_name):
    country_confirmed = country_data_name.groupby('ObservationDate').sum()['Confirmed'].reset_index()
    country_deaths = country_data_name.groupby('ObservationDate').sum()['Deaths'].reset_index()
    country_recovered = country_data_name.groupby('ObservationDate').sum()['Recovered'].reset_index()
    return country_confirmed, country_deaths, country_recovered

**Mortality and Recovery Factor**
* Mortality rate = (Number of Death Cases / Number of Confirmed Cases) x 100
* Recovery Rate= (Number of Recoverd Cases / Number of Confirmed Cases) x 100
* Mortality rate is showing a considerable for a pretty long time, which is positive sign
* Recovery Rate has started to pick up again which is a good sign, another supportive reason to why number of Closed Cases are increasing

**Growth Factor**
* Growth factor is the factor by which a quantity multiplies itself over time. The formula used is:
* Formula: Every day's new (Confirmed,Recovered,Deaths) / new (Confirmed,Recovered,Deaths) on the previous day.

In [ ]:
# function for find all rates of countries

def all_rate(country_datewise):
    country_datewise["Mortality Rate"]=(country_datewise["Deaths"]/country_datewise["Confirmed"])*100
    country_datewise["Recovery Rate"]=(country_datewise["Recovered"]/country_datewise["Confirmed"])*100
    country_datewise["Active Cases"]=country_datewise["Confirmed"]-country_datewise["Recovered"]-country_datewise["Deaths"]
    country_datewise["Closed Cases"]=country_datewise["Recovered"]+country_datewise["Deaths"]
    country_datewise['Growth Rate Acitve Case'] = (country_datewise["Confirmed"]-country_datewise["Recovered"]-country_datewise["Deaths"])/\
                                            (country_datewise["Confirmed"]-country_datewise["Recovered"]-country_datewise["Deaths"]).shift()
    country_datewise['Growth Rate Closed Case'] = (country_datewise["Recovered"]+country_datewise["Deaths"])/\
                                                (country_datewise["Recovered"]+country_datewise["Deaths"]).shift()
    return country_datewise

In [ ]:
def plot_country_bar(country_confirmed, country_death, country_recovered, country_name):

    trace1 = go.Bar(x = country_confirmed['ObservationDate'],
              y = country_confirmed['Confirmed'],
              name = 'Confirmed',
              marker_color = 'blue')

    trace2 = go.Bar(x = country_death['ObservationDate'],
              y = country_death['Deaths'],
              name = 'Death',
              marker_color = 'red')

    trace3 = go.Bar(x = country_recovered['ObservationDate'],
              y = country_recovered['Recovered'],
              name = 'Recovered',
              marker_color = 'green')

    layout = go.Layout(barmode = 'group',
                  bargap = 0.15,
                  bargroupgap = 0.1,
                  title = "{} Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)".format(country_name),
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    )

    data = [trace1, trace2, trace3]
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

In [ ]:
# function for countries line chart

def plot_country_line(country_confirmed, country_death, country_recovered, country_name):
    trace1 = go.Scatter(x = country_confirmed['ObservationDate'],
              y = country_confirmed['Confirmed'],
              name = 'Confirmed',
              mode='lines+markers',
              line=dict(color='blue', width=1))

    trace2 = go.Scatter(x = country_death['ObservationDate'],
              y = country_death['Deaths'],
              name = 'Death',
              mode='lines+markers',
              line=dict(color='red', width=1))

    trace3 = go.Scatter(x = country_recovered['ObservationDate'],
              y = country_recovered['Recovered'],
              name = 'Recovered',
              mode='lines+markers',
              line=dict(color='green', width=1))

    layout = go.Layout(title = "{} Corona Virus Cases - Confirmed, Deaths, Recovered (Line Chart)".format(country_name),
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    )

    data = [trace1, trace2, trace3]
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

In [ ]:
# function for mortlity and recovery rate for countries

def plot_country_mor_rec_rate(country_datewise_mor_rec, country_name):
    trace1 = go.Scatter(x = country_datewise_mor_rec.index,
              y = country_datewise_mor_rec['Mortality Rate'],
              name = 'Mortality Rate',
              mode='lines',
              line=dict(color='red', width=1.5))

    trace2 = go.Scatter(x = country_datewise_mor_rec.index,
              y = country_datewise_mor_rec['Recovery Rate'],
              name = 'Recovery Rate',
              mode='lines',
              line=dict(color='green', width=1.5))

    layout = go.Layout(title = "{} Corona Virus Cases - Mortality Rate ,Recovery Rate (Line Chart)".format(country_name),
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Rate',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    )

    data = [trace1, trace2]
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

In [ ]:
# function for ploting grwoth rate of country

def plot_country_growth_rate(country_datewise_growth, country_name):
    trace1 = go.Scatter(x = country_datewise_growth.index,
              y = country_datewise_growth['Growth Rate Acitve Case'],
              name = 'Growth Rate Acitve Case',
              mode='lines',
              line=dict(color='blue', width=1.5))

    trace2 = go.Scatter(x = country_datewise_growth.index,
              y = country_datewise_growth['Growth Rate Closed Case'],
              name = 'Growth Rate Closed Case',
              mode='lines',
              line=dict(color='green', width=1.5))

    layout = go.Layout(title = "{} Corona Virus Cases - Growth Rate Acitve Case and Closed Case (Line Chart)".format(country_name),
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Rate',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    )

    data = [trace1, trace2]
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

**india's covid case charts**

In [ ]:
# india data

india_confirmed, india_death, india_recoverd = country_all_data(country_data('India'))
india_datewise = country_data_datewise(country_data('India'))
india_rate = all_rate(india_datewise)

In [ ]:
# plot confirmed, death and recovery for india
plot_country_line(india_confirmed, india_death, india_recoverd, 'India')
plot_country_bar(india_confirmed, india_death, india_recoverd, 'India')

In [ ]:
# plot all rates for india

plot_country_mor_rec_rate(india_rate, 'India')
plot_country_growth_rate(india_rate, 'India')

**us's covid case chart**

In [ ]:
# us data

us_confirmed, us_death, us_recoverd = country_all_data(country_data('US'))
us_datewise = country_data_datewise(country_data('US'))
us_rate = all_rate(us_datewise)

In [ ]:
plot_country_line(us_confirmed, us_death, us_recoverd, 'US')
plot_country_bar(us_confirmed, us_death, us_recoverd, 'US')

In [ ]:
# plot all rates for india

plot_country_mor_rec_rate(us_rate, 'US')
plot_country_growth_rate(us_rate, 'US')

**Russia's covid case charts**

In [ ]:
russia_confirmed, russia_death, russia_recoverd = country_all_data(country_data('Russia'))
russia_datewise = country_data_datewise(country_data('Russia'))
russia_rate = all_rate(russia_datewise)

In [ ]:
plot_country_line(russia_confirmed, russia_death, russia_recoverd, 'Russia')
plot_country_bar(russia_confirmed, russia_death, russia_recoverd, 'Russia')

In [ ]:
plot_country_mor_rec_rate(russia_rate, 'Russia')
plot_country_growth_rate(russia_rate, 'Russia')

**Italy's covid case charts**

In [ ]:
Italy_confirmed, Italy_death, Italy_recoverd = country_all_data(country_data('Italy'))
Italy_datewise = country_data_datewise(country_data('Italy'))
Italy_rate = all_rate(Italy_datewise)

In [ ]:
plot_country_line(Italy_confirmed, Italy_death, Italy_recoverd, 'Italy')
plot_country_bar(Italy_confirmed, Italy_death, Italy_recoverd, 'Italy')

In [ ]:
plot_country_mor_rec_rate(Italy_rate, 'Italy')
plot_country_growth_rate(Italy_rate, 'Italy')

**Brazil's covid case charts**

In [ ]:
Brazil_confirmed, Brazil_death, Brazil_recoverd = country_all_data(country_data('Brazil'))
Brazil_datewise = country_data_datewise(country_data('Brazil'))
Brazil_rate = all_rate(Brazil_datewise)

In [ ]:
plot_country_line(Brazil_confirmed, Brazil_death, Brazil_recoverd, 'Brazil')
plot_country_bar(Brazil_confirmed, Brazil_death, Brazil_recoverd, 'Brazil')

In [ ]:
plot_country_mor_rec_rate(Brazil_rate, 'Brazil')
plot_country_growth_rate(Brazil_rate, 'Brazil')

**Spain's covid case charts**

In [ ]:
Spain_confirmed, Spain_death, Spain_recoverd = country_all_data(country_data('Spain'))
Spain_datewise = country_data_datewise(country_data('Spain'))
Spain_rate = all_rate(Spain_datewise)

In [ ]:
plot_country_line(Spain_confirmed, Spain_death, Spain_recoverd, 'Spain')
plot_country_bar(Spain_confirmed, Spain_death, Spain_recoverd, 'Spain')

In [ ]:
plot_country_mor_rec_rate(Spain_rate, 'Spain')
plot_country_growth_rate(Spain_rate, 'Spain')

# **Compare Countries**

In [ ]:
compare_countries = covid_data[covid_data["ObservationDate"]==covid_data["ObservationDate"].max()].\
                    groupby(["Country/Region"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'}).\
                    sort_values(["Confirmed"],ascending=False)
compare_countries["Mortality"] = (compare_countries["Deaths"]/compare_countries["Confirmed"])*100
compare_countries["Recovery"] = (compare_countries["Recovered"]/compare_countries["Confirmed"])*100
compare_countries["Active Cases"] = (compare_countries["Confirmed"]-compare_countries["Recovered"]-compare_countries["Deaths"])
compare_countries["Other Cases"] = (compare_countries["Recovered"]+compare_countries["Deaths"])

grouped_country = covid_data.groupby(["Country/Region","ObservationDate"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})
grouped_country["Active Cases"]=grouped_country["Confirmed"]-grouped_country["Recovered"]-grouped_country["Deaths"]
grouped_country["log_confirmed"]=np.log(grouped_country["Confirmed"])
grouped_country["log_active"]=np.log(grouped_country["Active Cases"])

In [ ]:
# last 24 hour data

country_last_24_confirmed=[]
country_last_24_recovered=[]
country_last_24_deaths=[]
for country in compare_countries.index:
    country_last_24_confirmed.append((grouped_country.loc[country].iloc[-1]-grouped_country.loc[country].iloc[-2])["Confirmed"])
    country_last_24_recovered.append((grouped_country.loc[country].iloc[-1]-grouped_country.loc[country].iloc[-2])["Recovered"])
    country_last_24_deaths.append((grouped_country.loc[country].iloc[-1]-grouped_country.loc[country].iloc[-2])["Deaths"])

In [ ]:
Last_24_Hours_country=pd.DataFrame(list(zip(compare_countries.index,country_last_24_confirmed,country_last_24_recovered,country_last_24_deaths)),
                                   columns=["Country Name","Last 24 Hours Confirmed","Last 24 Hours Recovered","Last 24 Hours Deaths"])

In [ ]:
Confirmed_last_24hr = Last_24_Hours_country.sort_values(["Last 24 Hours Confirmed"],ascending=False).head(15)
Recoverd_last_24hr = Last_24_Hours_country.sort_values(["Last 24 Hours Recovered"],ascending=False).head(15)
Deaths_last_24hr = Last_24_Hours_country.sort_values(["Last 24 Hours Deaths"],ascending=False).head(15)

In [ ]:
trace = go.Bar(x = Confirmed_last_24hr['Country Name'][:15],
              y = Confirmed_last_24hr['Last 24 Hours Confirmed'][:15],
              marker = dict(color='rgba(125, 215, 180, 0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with last 24 hours confirmed cases",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(Confirmed_last_24hr, values=Confirmed_last_24hr['Last 24 Hours Confirmed'], names=Confirmed_last_24hr['Country Name'],
             title='Confirmed cases last 24 hours Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
trace = go.Bar(x = Recoverd_last_24hr['Country Name'][:15],
              y = Recoverd_last_24hr['Last 24 Hours Recovered'][:15],
              marker = dict(color='rgba(255,155,128,0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with last 24 hours Recoverd cases",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Recoveries',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(Recoverd_last_24hr, values=Recoverd_last_24hr['Last 24 Hours Recovered'], names=Recoverd_last_24hr['Country Name'],
             title='Recoverd cases last 24 hours Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
trace = go.Bar(x = Deaths_last_24hr['Country Name'][:15],
              y = Deaths_last_24hr['Last 24 Hours Deaths'][:15],
              marker = dict(color='rgba(115, 155, 214, 0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with last 24 hours Deaths",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Deaths',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(Deaths_last_24hr, values=Deaths_last_24hr['Last 24 Hours Deaths'], names=Deaths_last_24hr['Country Name'],
             title='Death cases last 24 hours Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
confirmed_case_total =  compare_countries.sort_values(["Confirmed"],ascending=False)
trace = go.Bar(x = confirmed_case_total.index[:15],
              y = confirmed_case_total['Confirmed'][:15],
              marker = dict(color='rgba(125, 215, 180, 0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with total no. of confirmed cases",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(confirmed_case_total, values=confirmed_case_total['Confirmed'], names=confirmed_case_total.index,
             title='Confirmed cases Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter(
    x=confirmed_case_total.index[:15],
    y=confirmed_case_total['Confirmed'][:15],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(confirmed_case_total['Confirmed'][0:15]/25000),
        showscale=True
        )
)])

fig.update_layout(
    title='Most 15 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    template='plotly_white'
)
fig.show()

In [ ]:
death_case_total =  compare_countries.sort_values(["Deaths"],ascending=False)

trace = go.Bar(x = death_case_total.index[:15],
              y = death_case_total['Deaths'][:15],
              marker = dict(color='rgba(255,155,128,0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with total no. of deaths",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Recoveries',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(death_case_total, values=death_case_total['Deaths'], names=death_case_total.index,
             title='Death cases Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter(
    x=death_case_total.index[:10],
    y=death_case_total['Deaths'][:10],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(death_case_total['Deaths'][:10]/800),
        showscale=True
        )
)])

fig.update_layout(
    title='Most 10 Countries with Death cases',
    xaxis_title="Countries",
    yaxis_title="Death Cases",
    template='plotly_white'
)
fig.show()

In [ ]:
active_case_total =  compare_countries.sort_values(["Active Cases"],ascending=False)
trace = go.Bar(x = active_case_total.index[:15],
              y = active_case_total['Active Cases'][:15],
              marker = dict(color='rgba(125, 215, 180, 0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with total no. of Active cases",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(active_case_total, values=active_case_total['Active Cases'], names=active_case_total.index,
             title='Active cases Ratio',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter(
    x=active_case_total.index[:10],
    y=active_case_total['Active Cases'][:10],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(active_case_total['Active Cases'][:10]/15000),
        showscale=True
        )
)])

fig.update_layout(
    title='Most 10 Countries with Active cases',
    xaxis_title="Countries",
    yaxis_title="Active Cases",
    template='plotly_white'
)
fig.show()

In [ ]:
other_case_total =  compare_countries.sort_values(["Other Cases"],ascending=False)
trace = go.Bar(x = other_case_total.index[:15],
              y = other_case_total['Other Cases'][:15],
              marker = dict(color='rgba(150, 200, 100, 0.5)',
              line = dict(color='rgb(0,0,0)', width=1.5)))

layout = go.Layout(title="Top 15 countries with total no. of Other cases",
                  xaxis=dict(title='Country Name',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis=dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font=dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)')
)
data = [trace]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
fig = px.pie(other_case_total, values=other_case_total['Other Cases'], names=other_case_total.index,
             title='Other cases',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter(
    x=other_case_total.index[:15],
    y=other_case_total['Other Cases'][:15],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(other_case_total['Other Cases'][:15]/12000),
        showscale=True
        )
)])

fig.update_layout(
    title='Most 15 Countries with Other cases',
    xaxis_title="Countries",
    yaxis_title="Other Cases",
    template='plotly_white'
)
fig.show()

In [ ]:
# confirmed case comparision

trace1 = go.Scatter(x = india_confirmed['ObservationDate'],
              y = india_confirmed['Confirmed'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_confirmed['ObservationDate'],
              y = us_confirmed['Confirmed'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_confirmed['ObservationDate'],
              y = russia_confirmed['Confirmed'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_confirmed['ObservationDate'],
              y = Italy_confirmed['Confirmed'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_confirmed['ObservationDate'],
              y = Brazil_confirmed['Confirmed'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_confirmed['ObservationDate'],
              y = Spain_confirmed['Confirmed'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Confirmed cases comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# death case comparision

trace1 = go.Scatter(x = india_death['ObservationDate'],
              y = india_death['Deaths'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_death['ObservationDate'],
              y = us_death['Deaths'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_death['ObservationDate'],
              y = russia_death['Deaths'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_death['ObservationDate'],
              y = Italy_death['Deaths'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_death['ObservationDate'],
              y = Brazil_death['Deaths'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_death['ObservationDate'],
              y = Spain_death['Deaths'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Death cases comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# recoverd case comparision

trace1 = go.Scatter(x = india_recoverd['ObservationDate'],
              y = india_recoverd['Recovered'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_recoverd['ObservationDate'],
              y = us_recoverd['Recovered'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_recoverd['ObservationDate'],
              y = russia_recoverd['Recovered'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_recoverd['ObservationDate'],
              y = Italy_recoverd['Recovered'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_recoverd['ObservationDate'],
              y = Brazil_recoverd['Recovered'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_recoverd['ObservationDate'],
              y = Spain_recoverd['Recovered'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Recoverd cases comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='No. of Cases',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# Growth rate comparision

trace1 = go.Scatter(x = india_rate.index,
              y = india_rate['Growth Rate Acitve Case'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_rate.index,
              y = us_rate['Growth Rate Acitve Case'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_rate.index,
              y = russia_rate['Growth Rate Acitve Case'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_rate.index,
              y = Italy_rate['Growth Rate Acitve Case'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_rate.index,
              y = Brazil_rate['Growth Rate Acitve Case'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_rate.index,
              y = Spain_rate['Growth Rate Acitve Case'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Growth Rate Active Case comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Rate',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# mortality rate comparision

trace1 = go.Scatter(x = india_rate.index,
              y = india_rate['Mortality Rate'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_rate.index,
              y = us_rate['Mortality Rate'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_rate.index,
              y = russia_rate['Mortality Rate'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_rate.index,
              y = Italy_rate['Mortality Rate'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_rate.index,
              y = Brazil_rate['Mortality Rate'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_rate.index,
              y = Spain_rate['Mortality Rate'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Mortality Rate comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Rate',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# recovery rate comparision

trace1 = go.Scatter(x = india_rate.index,
              y = india_rate['Recovery Rate'],
              name = 'India',
              mode='lines')

trace2 = go.Scatter(x = us_rate.index,
              y = us_rate['Recovery Rate'],
              name = 'US',
              mode='lines')

trace3 = go.Scatter(x = russia_rate.index,
              y = russia_rate['Recovery Rate'],
              name = 'Russia',
              mode='lines')

trace4 = go.Scatter(x = Italy_rate.index,
              y = Italy_rate['Recovery Rate'],
              name = 'Italy',
              mode='lines')

trace5 = go.Scatter(x = Brazil_rate.index,
              y = Brazil_rate['Recovery Rate'],
              name = 'Brazil',
              mode='lines')

trace6 = go.Scatter(x = Spain_rate.index,
              y = Spain_rate['Recovery Rate'],
              name = 'Spain',
              mode='lines')

layout = go.Layout(title="Recovery Rate comparison plot",
                  xaxis = dict(title='Month',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Rate',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1, trace2, trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

# **If you found this kernel helpful, please upvote it**